In [2]:
import sqlite3
from database_handler import db_read
from fer_group import Fer_group
from tqdm import tqdm
import networkx as nx
import pickle

In [28]:
# Connect to the SQLite database
conn = sqlite3.connect('databases/all_graphs.db')
cursor = conn.cursor()

# Create checkpoint table with a primary key to ensure only one row exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS checkpoint (
    id INTEGER PRIMARY KEY,
    last_id INTEGER
)
""")
conn.commit()

# Fetch the last processed ID from the checkpoint table
cursor.execute("SELECT last_id FROM checkpoint WHERE id = 1")
row = cursor.fetchone()
last_id = row[0] if row else 0

print("Resuming from ID:", last_id)

# Get the total number of rows to process
cursor.execute("SELECT COUNT(*) FROM graphs WHERE id > ?", (last_id,))
total_rows = cursor.fetchone()[0]

print(f"Total rows to process: {total_rows}")

# Fetch all rows into memory starting from last processed ID
cursor.execute("SELECT id, graph_g6 FROM graphs WHERE id > ? ORDER BY id", (last_id,))
rows = cursor.fetchall()

progress_bar = tqdm(total=total_rows, initial=0, desc="Processing Rows")

# Process each row
for row in rows:
    current_id, graph_g6 = row

    # Convert G6 string to networkx graph
    G = nx.from_graph6_bytes(graph_g6.encode())

    # Compute Fer_group(G)
    fer = Fer_group(G)

    # Serialize the result
    s_fer = pickle.dumps(fer)

    # Update the row with the computed result
    cursor.execute("UPDATE graphs SET fer = ? WHERE id = ?", (s_fer, current_id))
    
    # Update the checkpoint and commit immediately
    cursor.execute("INSERT OR REPLACE INTO checkpoint (id, last_id) VALUES (1, ?)", (current_id,))
    conn.commit()  # Ensure checkpoint and data are saved after each row  

    # Update the progress bar
    progress_bar.update(1)

# Close the progress bar and connection
progress_bar.close()
conn.close()

Resuming from ID: 506
Total rows to process: 173036


Processing Rows:   0%|          | 505/173542 [00:52<4:58:49,  9.65it/s]


KeyboardInterrupt: 